In [1]:
import os
import lancedb
from sentence_transformers import SentenceTransformer
print("completed imports")



completed imports


In [2]:
# Function to split text into chunks
def chunk_text(text, chunk_size=200, overlap=50):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = " ".join(words[i:i + chunk_size])
        chunks.append(chunk)
    return chunks

# Step 1: Connect to LanceDB
db_path = "./lancedb_folder"
db = lancedb.connect(db_path)

# Step 2: Read text files
folder_path = r"../DataBase/documents"
files = [f for f in os.listdir(folder_path)]
# Step 3: Prepare data with chunking
data = []
for file in files:
    file_path = os.path.join(folder_path, file)
    # file_path = file
    with open(file_path, 'r', encoding='utf-8') as f:
        text_content = f.read()
        chunks = chunk_text(text_content)  # Chunk the text
        for i, chunk in enumerate(chunks):
            data.append({"filename": file, "chunk_id": i, "content": chunk})
print("done with chunking")
# Step 4: Generate embeddings for chunks
model = SentenceTransformer('all-MiniLM-L6-v2')  # Choose an appropriate model
texts = [entry['content'] for entry in data]
embeddings = model.encode(texts)

# Add embeddings to the data
for i, entry in enumerate(data):
    entry['vector'] = embeddings[i]
print(data)
# Step 5: Store in LanceDB
table_name = "chunked_text_files"
if table_name in db.table_names():
    table = db.open_table(table_name)
    table.add(data)
else:
    table = db.create_table(table_name, data=data)

print(f"Chunked data has been stored in LanceDB at: {db_path}")



done with chunking


c:\Users\vishn\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


[{'filename': 'pathway_readme.txt', 'chunk_id': 0, 'content': "Pathway is an open framework for high-throughput and low-latency real-time data processing. It is used to create Python code which seamlessly combines batch processing, streaming, and real-time API's for LLM apps. Pathway's distributed runtime (🦀-🐍) provides fresh results of your data pipelines whenever new inputs and requests are received. In the first place, Pathway was designed to be a life-saver (or at least a time-saver) for Python developers and ML/AI engineers faced with live data sources, where you need to react quickly to fresh data. Still, Pathway is a powerful tool that can be used for a lot of things. If you want to do streaming in Python, build an AI data pipeline, or if you are looking for your next Python data processing framework, keep reading. Pathway provides a high-level programming interface in Python for defining data transformations, aggregations, and other operations on data streams. With Pathway, you

In [ ]:
# db.drop_table("chunked_text_files")

In [5]:
import lancedb
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')  # Choose an appropriate model

db_path = "./lancedb_folder"
db = lancedb.connect(db_path)

table=db.open_table('chunked_text_files')


# number of rows
table.count_rows()

c:\Users\vishn\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


9

In [3]:
# get columns of the lancedb table
table.schema

filename: string
chunk_id: int64
content: string
vector: fixed_size_list<item: float>[384]
  child 0, item: float

In [ ]:
# Step 6: Query the database (e.g., Vector Search)
query = "Explain the purpose of Pathway."
query_embedding = model.encode([query])[0]
results = table.search(query_embedding).limit(10).to_pandas()
print(list(results["content"]))
# print("Top matching chunks:")
# print(results[["filename", "chunk_id", "content"]])


9


In [ ]:
"""
# convert my data to json format and save it to a file
import json
import numpy as np

# Convert numpy arrays to lists
for entry in data:
    entry['vector'] = entry['vector'].tolist()

with open('data.json', 'w') as f:
    json.dump(data, f)
"""
